
## IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report
### Opening a New Shopping Mall in Lagos, Nigeria
Build a dataframe of LGA's in Lagos, Nigeria by web scraping the data from Wikipedia page

Get the geographical coordinates of the neighborhoods

Obtain the venue data for the LGA's from Foursquare API

Explore and cluster the LGA's

Select the best cluster to open a new shopping mall

##### Import libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge geocoder --yes # uncomment this line if you haven't completed the Foursquare API lab
import geocoder # to get coordinates

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

    geocoder: 1.38.1-py_1 conda-forge
    ratelim:  0.1.6-py_2  conda-forge


geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing tran

In [4]:
url='https://en.wikipedia.org/wiki/List_of_Lagos_State_local_government_areas_by_population'
results = pd.read_html(url)

In [16]:
df=results[2]
df

,Rank,LGA,Population
0,1.0,Alimosho,5700714
1,2.0,Ajeromi-Ifelodun,1746634
2,3.0,Kosofe,665421
3,4.0,Mushin,633543
4,5.0,Oshodi-Isolo,10621789
5,6.0,Ojo,598332
6,7.0,Ikorodu,535811
7,8.0,Surulere,504409
8,9.0,Agege,461123
9,10.0,Ifako-Ijaiye,428812


In [19]:
# Select only the LGA column and skip the last column
df_lga=df[['LGA']].head(20)
df_lga.columns = ['Area']
df_lga

,Area
0,Alimosho
1,Ajeromi-Ifelodun
2,Kosofe
3,Mushin
4,Oshodi-Isolo
5,Ojo
6,Ikorodu
7,Surulere
8,Agege
9,Ifako-Ijaiye


### Geographical cordinates

In [20]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Local Government Areas, Lagos'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [21]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in df_lga["Area"].tolist() ]

In [22]:
coords

[[6.605860000000064, 3.3091400000000704],
 [6.446440000000052, 3.3402700000000323],
 [6.599610000000041, 3.383630000000039],
 [6.527820000000077, 3.3483300000000327],
 [6.562360000000069, 3.3451400000000717],
 [6.461260000000038, 3.14474000000007],
 [6.623090000000047, 3.4812400000000707],
 [6.500290000000064, 3.353990000000067],
 [6.6227700000000596, 3.306700000000035],
 [6.644160000000056, 3.3259400000000596],
 [6.532520000000034, 3.381520000000023],
 [6.469960000000071, 3.2835900000000606],
 [6.524260000000027, 3.3765900000000215],
 [6.595920000000035, 3.3376800000000344],
 [6.43318000000005, 3.439900000000023],
 [6.4198200000000725, 2.7627900000000523],
 [6.43763000000007, 3.3672300000000632],
 [6.458930000000066, 3.382350000000031],
 [6.674310000000048, 3.712030000000027],
 [6.473160000000064, 3.844630000000052]]

In [23]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [24]:
# merge the coordinates into the original dataframe
df_lga['Latitude'] = df_coords['Latitude']
df_lga['Longitude'] = df_coords['Longitude']

In [25]:
# check the neighborhoods and the coordinates
print(df_lga.shape)
df_lga

(20, 3)


,Area,Latitude,Longitude
0,Alimosho,6.60586,3.30914
1,Ajeromi-Ifelodun,6.44644,3.34027
2,Kosofe,6.59961,3.38363
3,Mushin,6.52782,3.34833
4,Oshodi-Isolo,6.56236,3.34514
5,Ojo,6.46126,3.14474
6,Ikorodu,6.62309,3.48124
7,Surulere,6.50029,3.35399
8,Agege,6.62277,3.30670
9,Ifako-Ijaiye,6.64416,3.32594


In [26]:
# The code was removed by Watson Studio for sharing.

{'file_name': 'lagos_lga_coor.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'courseramachineleaeningproject-donotdelete-pr-fmwp3mbycu6qam',
 'asset_id': '863bbf60-d01e-4a91-b18f-902a42578a95'}

#### Create a map of Lagos, Nigeria with neighborhoods superimposed on top

In [27]:
# get the coordinates of Kuala Lumpur
address = 'Lagos, Nigeria'

geolocator = Nominatim(user_agent="lagos_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lagos, Nigeria {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lagos, Nigeria 6.4550575, 3.3941795.


In [31]:
# create map of Lagos using latitude and longitude values
map_lagos = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df_lga['Latitude'], df_lga['Longitude'], df_lga['Area']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_lagos)  
    
map_lagos

In [32]:
# save the map as HTML file
#map_lagos.save('map_lagos.html')

#### Using the Foursquare API to explore the neighborhoods

In [34]:
"""# define Foursquare Credentials and Version
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)"""

"# define Foursquare Credentials and Version\nCLIENT_ID = 'your Foursquare ID' # your Foursquare ID\nCLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret\nVERSION = '20180605' # Foursquare API version\n\nprint('Your credentails:')\nprint('CLIENT_ID: ' + CLIENT_ID)\nprint('CLIENT_SECRET:' + CLIENT_SECRET)"

In [33]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: TIHOWSNZBSDE4BXCSJPNDNMLZMIV235I0JFFVRSAKSBVUPTA
CLIENT_SECRET:YTHZRJ4CUPBXBOAVOH1KGKEDQZZFMWQBX3I51H4RXBTFYLZX


#### Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [147]:
radius = 4000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df_lga['Latitude'], df_lga['Longitude'], df_lga['Area']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [148]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Area', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(701, 7)


,Area,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alimosho,6.60586,3.30914,KFC,6.604590,3.308936,Fast Food Restaurant
1,Alimosho,6.60586,3.30914,Pink Ice Boutique,6.601810,3.301896,Boutique
2,Alimosho,6.60586,3.30914,"National Stadium, Surulere",6.593562,3.304300,Athletics & Sports
3,Alimosho,6.60586,3.30914,Wings Restaurant,6.576023,3.325612,Restaurant
4,Alimosho,6.60586,3.30914,"ICE CREAM PLUS, Medical Road, Ikeja.",6.596598,3.340001,Soup Place


#### Let's check how many venues were returned for each neighorhood

In [149]:
venues_df.groupby(["Area"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Area,,,,,,
Agege,23,23,23,23,23,23
Ajeromi-Ifelodun,7,7,7,7,7,7
Alimosho,36,36,36,36,36,36
Amuwo-Odofin,20,20,20,20,20,20
Apapa,25,25,25,25,25,25
Eti-Osa,100,100,100,100,100,100
Ibeju-Lekki,1,1,1,1,1,1
Ifako-Ijaiye,23,23,23,23,23,23
Ikeja,82,82,82,82,82,82


#### Let's find out how many unique categories can be curated from all the returned venues

In [150]:

print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 113 uniques categories.


In [151]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

In [152]:
# print out the list of categories
hi=venues_df['VenueCategory'].unique()
hi=hi.tolist()
hi.sort()
hi

['African Restaurant',
 'Airport',
 'Airport Food Court',
 'Airport Lounge',
 'Airport Terminal',
 'American Restaurant',
 'Arcade',
 'Art Gallery',
 'Arts & Entertainment',
 'Asian Restaurant',
 'Athletics & Sports',
 'Auto Garage',
 'BBQ Joint',
 'Bagel Shop',
 'Bakery',
 'Bar',
 'Baseball Stadium',
 'Basketball Court',
 'Beach',
 'Beach Bar',
 'Beer Garden',
 'Bike Rental / Bike Share',
 'Bistro',
 'Boutique',
 'Brazilian Restaurant',
 'Breakfast Spot',
 'Burger Joint',
 'Bus Station',
 'Bus Stop',
 'Café',
 'Campground',
 'Chinese Restaurant',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'Convenience Store',
 'Convention Center',
 'Cosmetics Shop',
 'Cupcake Shop',
 'Department Store',
 'Dessert Shop',
 'Diner',
 'Electronics Store',
 'Farmers Market',
 'Fast Food Restaurant',
 'Flea Market',
 'Food',
 'Food & Drink Shop',
 'Food Court',
 'Fried Chicken Joint',
 'Frozen Yogurt Shop',
 'Golf Course',
 'Grocery Store',
 'Gym',
 'Gym / Fitness Center',
 'Harbor / Marina',
 'He

In [153]:
#Count of Shopping Mall per Area
venues_df[venues_df["VenueCategory"]== "Shopping Mall"].groupby(["Area"]).count()["VenueCategory"]

Area
Agege               1
Ajeromi-Ifelodun    1
Alimosho            1
Amuwo-Odofin        1
Apapa               2
Eti-Osa             1
Ifako-Ijaiye        1
Ikeja               3
Kosofe              3
Lagos Island        3
Lagos Mainland      3
Mushin              4
Oshodi-Isolo        5
Somolu              4
Surulere            4
Name: VenueCategory, dtype: int64

#### Analyze Each Neighborhood

In [154]:
# one hot encoding
lag_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
lag_onehot['Area'] = venues_df['Area'] 

# move neighborhood column to the first column
fixed_columns = [lag_onehot.columns[-1]] + list(lag_onehot.columns[:-1])
lag_onehot = lag_onehot[fixed_columns]

print(lag_onehot.shape)
lag_onehot.head()

(701, 114)


,Area,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arcade,Art Gallery,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Basketball Court,Beach,Beach Bar,Beer Garden,Bike Rental / Bike Share,Bistro,Boutique,Brazilian Restaurant,Breakfast Spot,Burger Joint,Bus Station,Bus Stop,Café,Campground,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Diner,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Heliport,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Karaoke Bar,Kebab Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Music Venue,Neighborhood,Nightclub,Noodle House,Optical Shop,Park,Performing Arts Venue,Pharmacy,Photography Studio,Pier,Pizza Place,Playground,Plaza,Pool,Pub,Rental Car Location,Resort,Restaurant,Salon / Barbershop,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Soup Place,Spa,Sports Bar,Stadium,Steakhouse,Tennis Court,Theater,Theme Restaurant,Toll Plaza,Train Station,Vineyard,Wine Bar,Wine Shop
0,Alimosho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alimosho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alimosho,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alimosho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Alimosho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [155]:
lag_grouped = lag_onehot.groupby(["Area"]).mean().reset_index()

print(lag_grouped.shape)
lag_grouped

(18, 114)


,Area,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arcade,Art Gallery,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Basketball Court,Beach,Beach Bar,Beer Garden,Bike Rental / Bike Share,Bistro,Boutique,Brazilian Restaurant,Breakfast Spot,Burger Joint,Bus Station,Bus Stop,Café,Campground,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Diner,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Heliport,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Karaoke Bar,Kebab Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Music Venue,Neighborhood,Nightclub,Noodle House,Optical Shop,Park,Performing Arts Venue,Pharmacy,Photography Studio,Pier,Pizza Place,Playground,Plaza,Pool,Pub,Rental Car Location,Resort,Restaurant,Salon / Barbershop,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Soup Place,Spa,Sports Bar,Stadium,Steakhouse,Tennis Court,Theater,Theme Restaurant,Toll Plaza,Train Station,Vineyard,Wine Bar,Wine Shop
0,Agege,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.00,0.00,0.000000,0.00,0.00,0.043478,0.00,0.000000,0.000000,0.173913,0.00,0.000000,0.043478,0.000000,0.000000,0.00,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.043478,0.260870,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.043478,0.043478,0.000000,0.000000,0.00,0.00,0.043478,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.043478,0.000000,0.000000,0.000000,0.00,0.043478,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.043478,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000
1,Ajeromi-Ifelodun,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.00,0.000000,0.000000,0.000000,0.142857,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.142857,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.142857,0.142857,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.142857,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000
2,Alimosho,0.000000,0.055556,0.027778,0.027778,0.000000,0.027778,0.00,0.000000,0.000000,0.00,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.00,0.00,0.000000,0.00,0.00,0.027778,0.00,0.027778,0.000000,0.083333,0.00,0.000000,0.027778,0.000000,0.000000,0.00,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.027778,0.000000,0.166667,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.027778,0.000000,0.027778,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.027778,0.055556,0.000000,0.00,0.00,0.027778,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.027778,0.000000,0.000000,0.000000,0.00,0.

In [156]:
len(lag_grouped[lag_grouped["Shopping Mall"] > 0])

15

#### Create a new DataFrame for Shopping Mall data only

In [157]:
lag_mall = lag_grouped[["Area","Shopping Mall"]]
lag_mall

,Area,Shopping Mall
0,Agege,0.043478
1,Ajeromi-Ifelodun,0.142857
2,Alimosho,0.027778
3,Amuwo-Odofin,0.050000
4,Apapa,0.080000
5,Eti-Osa,0.010000
6,Ibeju-Lekki,0.000000
7,Ifako-Ijaiye,0.043478
8,Ikeja,0.036585
9,Ikorodu,0.000000


#### Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Lagos into 3 clusters.

In [158]:
# set number of clusters
kclusters = 3

lag_clustering = lag_mall.drop(["Area"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lag_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 0, 2, 1, 1, 0, 0, 1], dtype=int32)

In [159]:

# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
lag_merged = lag_mall.copy()

# add clustering labels
lag_merged["Cluster Labels"] = kmeans.labels_

In [160]:
lag_merged.rename(columns={"Area": "Area"}, inplace=True)
lag_merged.head()

,Area,Shopping Mall,Cluster Labels
0,Agege,0.043478,0
1,Ajeromi-Ifelodun,0.142857,2
2,Alimosho,0.027778,0
3,Amuwo-Odofin,0.050000,0
4,Apapa,0.080000,2


In [161]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
lag_merged = lag_merged.join(df_lga.set_index("Area"), on="Area")

print(lag_merged.shape)
lag_merged.head() 

(18, 5)


,Area,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Agege,0.043478,0,6.62277,3.30670
1,Ajeromi-Ifelodun,0.142857,2,6.44644,3.34027
2,Alimosho,0.027778,0,6.60586,3.30914
3,Amuwo-Odofin,0.050000,0,6.46996,3.28359
4,Apapa,0.080000,2,6.43763,3.36723


In [162]:
#sort the results by Cluster Labels
print(lag_merged.shape)
lag_merged.sort_values(["Cluster Labels"], inplace=True)
lag_merged

(18, 5)


,Area,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Agege,0.043478,0,6.62277,3.30670
15,Oshodi-Isolo,0.053763,0,6.56236,3.34514
12,Lagos Mainland,0.058824,0,6.52426,3.37659
10,Kosofe,0.045455,0,6.59961,3.38363
7,Ifako-Ijaiye,0.043478,0,6.64416,3.32594
8,Ikeja,0.036585,0,6.59592,3.33768
3,Amuwo-Odofin,0.050000,0,6.46996,3.28359
2,Alimosho,0.027778,0,6.60586,3.30914
6,Ibeju-Lekki,0.000000,1,6.47316,3.84463
9,Ikorodu,0.000000,1,6.62309,3.48124


#### Finally, let's visualize the resulting clusters

In [163]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lag_merged['Latitude'], lag_merged['Longitude'], lag_merged['Area'], lag_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

##### Cluster 0

In [164]:
lag_merged.loc[lag_merged['Cluster Labels'] == 0]

,Area,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Agege,0.043478,0,6.62277,3.30670
15,Oshodi-Isolo,0.053763,0,6.56236,3.34514
12,Lagos Mainland,0.058824,0,6.52426,3.37659
10,Kosofe,0.045455,0,6.59961,3.38363
7,Ifako-Ijaiye,0.043478,0,6.64416,3.32594
8,Ikeja,0.036585,0,6.59592,3.33768
3,Amuwo-Odofin,0.050000,0,6.46996,3.28359
2,Alimosho,0.027778,0,6.60586,3.30914


##### Cluster 1

In [165]:
lag_merged.loc[lag_merged['Cluster Labels'] == 1]

,Area,Shopping Mall,Cluster Labels,Latitude,Longitude
6,Ibeju-Lekki,0.00,1,6.47316,3.84463
9,Ikorodu,0.00,1,6.62309,3.48124
14,Ojo,0.00,1,6.46126,3.14474
5,Eti-Osa,0.01,1,6.43318,3.43990


##### Cluster 2

In [166]:
lag_merged.loc[lag_merged['Cluster Labels'] == 2]

,Area,Shopping Mall,Cluster Labels,Latitude,Longitude
4,Apapa,0.080000,2,6.43763,3.36723
16,Somolu,0.097561,2,6.53252,3.38152
11,Lagos Island,0.088235,2,6.45893,3.38235
13,Mushin,0.090909,2,6.52782,3.34833
1,Ajeromi-Ifelodun,0.142857,2,6.44644,3.34027
17,Surulere,0.080000,2,6.50029,3.35399


##### Observations:
Most of the shopping malls are concentrated in the central area of Lagos, with the highest number in cluster 0 and moderate number in cluster 2. On the other hand, cluster 1 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls and the area is also witnessing high population growth. Meanwhile, shopping malls in cluster 0 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping mall in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 2 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 0 which already have high concentration of shopping malls and suffering from intense competition.